# Kaggle Bike Sharing Demand(レンタサイクルの需要予測)

## データの確認

In [594]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt # グラフ描画用
import japanize_matplotlib
import seaborn as sns; sns.set() # グラフ描画用

In [595]:
train = pd.read_csv('input/train.csv')
test = pd.read_csv('input/test.csv')
df = pd.concat([train, test])
df = df.reset_index(drop=True)
# print(len(df))
# test.shape

In [596]:
# def y_transform(df, col):
#     df[col] = np.sqrt(df[col])
#     # count_min  = df.loc[df[col].values > 0, col].min() #0を除いた最小値
#     # df[col] = train[col] + count_min
#     # df[col] = np.log(df[col])*10
    
# y_transform(train, 'casual')
# y_transform(train, 'registered')
# y_transform(train, 'count')

# fig=plt.figure(figsize=(20,5))

# def plt_setting(ax, trg, num, color):
#     ax = fig.add_subplot(1,3,num)
#     bin_width = 1
#     plt.xlim(0,50)
#     plt.ylim(0,1000)
#     plt.tick_params(labelsize=10)
#     plt.ylabel('freq',fontsize=16)
#     plt.xlabel(trg,fontsize=16)
#     ax.hist(train[trg], bins=range(0,1001,bin_width), alpha=1, color=color, ec=color)

# plt_setting(ax1, 'casual',     1, 'r')
# plt_setting(ax2, 'registered', 2, 'b')
# plt_setting(ax3, 'count',      3, 'g')

# plt.show()

In [597]:
test_datetime = pd.DataFrame(test['datetime'])
test_datetime.head()

,datetime
0,2011-01-20 00:00:00
1,2011-01-20 01:00:00
2,2011-01-20 02:00:00
3,2011-01-20 03:00:00
4,2011-01-20 04:00:00


In [598]:
display(train.shape)
display(test.shape)
display(df.shape)
print('test:' + str(round(test.shape[0]/df.shape[0]*100,1)) + '%')

(10886, 12)

(6493, 9)

(17379, 12)

test:37.4%


In [599]:
train.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


In [600]:
test.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed
0,2011-01-20 00:00:00,1,0,1,1,10.66,11.365,56,26.0027
1,2011-01-20 01:00:00,1,0,1,1,10.66,13.635,56,0.0000
2,2011-01-20 02:00:00,1,0,1,1,10.66,13.635,56,0.0000
3,2011-01-20 03:00:00,1,0,1,1,10.66,12.880,56,11.0014
4,2011-01-20 04:00:00,1,0,1,1,10.66,12.880,56,11.0014


In [601]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10886 entries, 0 to 10885
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   datetime    10886 non-null  object 
 1   season      10886 non-null  int64  
 2   holiday     10886 non-null  int64  
 3   workingday  10886 non-null  int64  
 4   weather     10886 non-null  int64  
 5   temp        10886 non-null  float64
 6   atemp       10886 non-null  float64
 7   humidity    10886 non-null  int64  
 8   windspeed   10886 non-null  float64
 9   casual      10886 non-null  int64  
 10  registered  10886 non-null  int64  
 11  count       10886 non-null  int64  
dtypes: float64(3), int64(8), object(1)
memory usage: 1020.7+ KB


In [602]:
df.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,10886.000000,10886.000000,10886.000000
mean,2.501640,0.028770,0.682721,1.425283,20.376474,23.788755,62.722884,12.736540,36.021955,155.552177,191.574132
std,1.106918,0.167165,0.465431,0.639357,7.894801,8.592511,19.292983,8.196795,49.960477,151.039033,181.144454
min,1.000000,0.000000,0.000000,1.000000,0.820000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2.000000,0.000000,0.000000,1.000000,13.940000,16.665000,48.000000,7.001500,4.000000,36.000000,42.000000
50%,3.000000,0.000000,1.000000,1.000000,20.500000,24.240000,63.000000,12.998000,17.000000,118.000000,145.000000
75%,3.000000,0.000000,1.000000,2.000000,27.060000,31.060000,78.000000,16.997900,49.000000,222.000000,284.000000
max,4.000000,1.000000,1.000000,4.000000,41.000000,50.000000,100.000000,56.996900,367.000000,886.000000,977.000000


In [603]:
# season = 4
# display(((train["weather"].values == 1) & (train["season"].values == season)).sum() / (train["season"].values == season).sum()*100)
# display(((train["weather"].values == 2) & (train["season"].values == season)).sum() / (train["season"].values == season).sum()*100)
# display(((train["weather"].values == 3) & (train["season"].values == season)).sum() / (train["season"].values == season).sum()*100)
# display(((train["weather"].values == 4) & (train["season"].values == season)).sum() / (train["season"].values == season).sum()*100)
# season = 3
# train.loc[(train["season"].values == season)]['temp'].min()

## データの加工

やること
- 日付から年、月、日、時間の列を生成 ok
- season番号の振り直し ok
- weekend列の作成 ok
- 新規利用者比率
- 降水量の推定. weatherとhumidityで特徴量作れるか
- キャンペーン日の特定(月初めの決まった曜日？)

In [604]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['year'] = pd.to_datetime(df['datetime']).dt.strftime('%Y')
df['month'] = pd.to_datetime(df['datetime']).dt.strftime('%m')
df['day'] = pd.to_datetime(df['datetime']).dt.strftime('%d')
df['weekday'] = pd.to_datetime(df['datetime']).dt.strftime('%a')
df['hour'] = pd.to_datetime(df['datetime']).dt.strftime('%H')

In [605]:
df = df.sort_values('datetime')
window=24*30
df['casual_mov_ave'] = df['casual'].rolling(window=window, center=True).mean()
df['registered_mov_ave'] = df['registered'].rolling(window=window, center=True).mean()

In [606]:
df['weekend'] = 0
df.loc[(df['holiday'].values==0) & (df['workingday'].values==0),'weekend'] = 1

In [607]:
df['day_type'] = 'holiday'
df.loc[(df['workingday'].values==1), 'day_type'] = 'workingday'
df.loc[(df['weekend'].values==1), 'day_type'] = 'weekend'

In [608]:
df['month'] = df['month'].astype('int')
spring_begin = 3
summer_begin = 6
fall_begin = 9
df['season_rev'] = 4 #12,1,2月は面倒なので先に代入
df.loc[(df['month'].values >= spring_begin) & (df['month'].values < (spring_begin+3)),'season_rev'] = 1
df.loc[(df['month'].values >= summer_begin) & (df['month'].values < (summer_begin+3)),'season_rev'] = 2
df.loc[(df['month'].values >= fall_begin  ) & (df['month'].values < (fall_begin+3  )),'season_rev'] = 3
df.head()

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,...,year,month,day,weekday,hour,casual_mov_ave,registered_mov_ave,weekend,day_type,season_rev
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3.0,...,2011,1,01,Sat,00,NaN,NaN,1,weekend,4
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8.0,...,2011,1,01,Sat,01,NaN,NaN,1,weekend,4
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5.0,...,2011,1,01,Sat,02,NaN,NaN,1,weekend,4
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3.0,...,2011,1,01,Sat,03,NaN,NaN,1,weekend,4
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0.0,...,2011,1,01,Sat,04,NaN,NaN,1,weekend,4


In [609]:
df['casual_ratio'] = df['casual']/df['count']*100

In [610]:
# df = df[['datetime', 'year', 'month', 'day', 'weekday', 'hour', 'season', 'season_rev',
#          'day_type', 'holiday', 'workingday', 'weekend',
#          'weather', 'temp','atemp', 'humidity', 'windspeed',
#          'casual', 'registered', 'count', 'casual_ratio',
#          'casual_mov_Ave', 'registered_mov_Ave'
#        ]]
# df.head()

In [611]:
df.tail(10)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,...,month,day,weekday,hour,casual_mov_ave,registered_mov_ave,weekend,day_type,season_rev,casual_ratio
17369,2012-12-31 14:00:00,1,0,1,2,11.48,13.635,45,15.0013,NaN,...,12,31,Mon,14,NaN,NaN,0,workingday,4,NaN
17370,2012-12-31 15:00:00,1,0,1,2,11.48,14.395,45,8.9981,NaN,...,12,31,Mon,15,NaN,NaN,0,workingday,4,NaN
17371,2012-12-31 16:00:00,1,0,1,2,10.66,12.880,48,12.9980,NaN,...,12,31,Mon,16,NaN,NaN,0,workingday,4,NaN
17372,2012-12-31 17:00:00,1,0,1,2,10.66,14.395,48,6.0032,NaN,...,12,31,Mon,17,NaN,NaN,0,workingday,4,NaN
17373,2012-12-31 18:00:00,1,0,1,2,10.66,13.635,48,8.9981,NaN,...,12,31,Mon,18,NaN,NaN,0,workingday,4,NaN
17374,2012-12-31 19:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,...,12,31,Mon,19,NaN,NaN,0,workingday,4,NaN
17375,2012-12-31 20:00:00,1,0,1,2,10.66,12.880,60,11.0014,NaN,...,12,31,Mon,20,NaN,NaN,0,workingday,4,NaN
17376,2012-12-31 21:00:00,1,0,1,1,10.66,12.880,60,11.0014,NaN,...,12,31,Mon,21,NaN,NaN,0,workingday,4,NaN
17377,2012-12-31 22:00:00,1,0,1,1,10.66,13.635,56,8.9981,NaN,...,12,31,Mon,22,NaN,NaN,0,workingday,4,NaN
17378,2012-12-31 23:00:00,1,0,1,1,10.66,13.635,65,8.9981,NaN,...,12,31,Mon,23,NaN,NaN,0,workingday,4,NaN


In [612]:
df.head(10)

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,...,month,day,weekday,hour,casual_mov_ave,registered_mov_ave,weekend,day_type,season_rev,casual_ratio
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0000,3.0,...,1,01,Sat,00,NaN,NaN,1,weekend,4,18.750000
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0000,8.0,...,1,01,Sat,01,NaN,NaN,1,weekend,4,20.000000
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0000,5.0,...,1,01,Sat,02,NaN,NaN,1,weekend,4,15.625000
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0000,3.0,...,1,01,Sat,03,NaN,NaN,1,weekend,4,23.076923
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0000,0.0,...,1,01,Sat,04,NaN,NaN,1,weekend,4,0.000000
5,2011-01-01 05:00:00,1,0,0,2,9.84,12.880,75,6.0032,0.0,...,1,01,Sat,05,NaN,NaN,1,weekend,4,0.000000
6,2011-01-01 06:00:00,1,0,0,1,9.02,13.635,80,0.0000,2.0,...,1,01,Sat,06,NaN,NaN,1,weekend,4,100.000000
7,2011-01-01 07:00:00,1,0,0,1,8.20,12.880,86,0.0000,1.0,...,1,01,Sat,07,NaN,NaN,1,weekend,4,33.333333
8,2011-01-01 08:00:00,1,0,0,1,9.84,14.395,75,0.0000,1.0,...,1,01,Sat,08,NaN,NaN,1,weekend,4,12.500000
9,2011-01-01 09:00:00,1,0,0,1,13.12,17.425,76,0.0000,8.0,...,1,01,Sat,09,NaN,NaN,1,weekend,4,57.142857


## データの前処理
時系列データをint型に変更
day_typeをLabelEncoderで数値に変換

In [613]:
from sklearn.preprocessing import LabelEncoder

In [614]:
df['year'] = df['year'].astype('int')
df['day'] = df['day'].astype('int')
df['hour'] = df['hour'].astype('int')

In [615]:
le = LabelEncoder()
le.fit(df['weekday'])
df['weekday'] = le.transform(df['weekday'])
le.fit(df['day_type'])
df['day_type'] = le.transform(df['day_type'])

In [616]:
# casualとregisteredを対数変換
# 0を含むため全データに1を足してから対数変換
# 予測値を変換した後に1を引く
# count_min = 1
# casual_max = df['casual'].max()
# registered_max = df['registered'].max()
# display(df.loc[df['casual'].values > 0, 'casual'].min())

# def log_transform(trg):
#     df[trg] = df[trg]/(df[trg].max()) #0~1になる
#     count_min = df.loc[df['casual'].values > 0, 'casual'].min()
#     df[trg] = df[trg] + count_min
#     display(df.head())
    

# log_transform('casual')
# log_transform('registered')

In [617]:
df.head()
df.describe()

,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,...,month,day,weekday,hour,casual_mov_ave,registered_mov_ave,weekend,day_type,season_rev,casual_ratio
count,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,17379.000000,10886.000000,10886.000000,...,17379.000000,17379.000000,17379.000000,17379.000000,0.0,0.0,17379.000000,17379.000000,17379.000000,10886.000000
mean,2.501640,0.028770,0.682721,1.425283,20.376474,23.788755,62.722884,12.736540,36.021955,155.552177,...,6.537775,15.683411,2.992577,11.546752,NaN,NaN,0.288509,1.653950,2.485241,17.100841
std,1.106918,0.167165,0.465431,0.639357,7.894801,8.592511,19.292983,8.196795,49.960477,151.039033,...,3.438776,8.789373,1.997928,6.914405,NaN,NaN,0.453082,0.532782,1.115985,13.692361
min,1.000000,0.000000,0.000000,1.000000,0.820000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,1.000000,0.000000
25%,2.000000,0.000000,0.000000,1.000000,13.940000,16.665000,48.000000,7.001500,4.000000,36.000000,...,4.000000,8.000000,1.000000,6.000000,NaN,NaN,0.000000,1.000000,1.000000,6.168831
50%,3.000000,0.000000,1.000000,1.000000,20.500000,24.240000,63.000000,12.998000,17.000000,118.000000,...,7.000000,16.000000,3.000000,12.000000,NaN,NaN,0.000000,2.000000,2.000000,14.473684
75%,3.000000,0.000000,1.000000,2.000000,27.060000,31.060000,78.000000,16.997900,49.000000,222.000000,...,10.000000,23.000000,5.000000,18.000000,NaN,NaN,1.000000,2.000000,3.000000,25.342466
max,4.000000,1.000000,1.000000,4.000000,41.000000,50.000000,100.000000,56.996900,367.000000,886.000000,...,12.000000,31.000000,6.000000,23.000000,NaN,NaN,1.000000,2.000000,4.000000,100.000000


In [618]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17379 entries, 0 to 17378
Data columns (total 23 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   datetime            17379 non-null  datetime64[ns]
 1   season              17379 non-null  int64         
 2   holiday             17379 non-null  int64         
 3   workingday          17379 non-null  int64         
 4   weather             17379 non-null  int64         
 5   temp                17379 non-null  float64       
 6   atemp               17379 non-null  float64       
 7   humidity            17379 non-null  int64         
 8   windspeed           17379 non-null  float64       
 9   casual              10886 non-null  float64       
 10  registered          10886 non-null  float64       
 11  count               10886 non-null  float64       
 12  year                17379 non-null  int64         
 13  month               17379 non-null  int64     

## 機械学習
trainデータを分割してcasualとregisteredを目的変数に設定

In [619]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error # モデル評価用(平均二乗誤差)
from sklearn.metrics import r2_score # モデル評価用(決定係数)

In [620]:
df = df[[
    # 'datetime', 
    'year', 
    'month', 
    # 'day', #train:1-19,test:20-
    'weekday', 
    'hour', 
    # 'season',       
    # 'season_rev', 
    'day_type', 
    # 'holiday', 
    # 'workingday', 
    # 'weekend', 
    'weather',
    'temp',
    # 'atemp',
    'humidity',
    'windspeed',
    'casual',
    'registered',
    'casual_mov_ave',
    'registered_mov_ave',
    # 'count',
    # 'casual_ratio',
    ]]
df.head()

,year,month,weekday,hour,day_type,weather,temp,humidity,windspeed,casual,registered,casual_mov_ave,registered_mov_ave
0,2011,1,2,0,1,1,9.84,81,0.0,3.0,13.0,NaN,NaN
1,2011,1,2,1,1,1,9.02,80,0.0,8.0,32.0,NaN,NaN
2,2011,1,2,2,1,1,9.02,80,0.0,5.0,27.0,NaN,NaN
3,2011,1,2,3,1,1,9.84,75,0.0,3.0,10.0,NaN,NaN
4,2011,1,2,4,1,1,9.84,75,0.0,0.0,1.0,NaN,NaN


In [621]:
df = df.sort_index()
train = df.iloc[:len(train),:].copy()
test  = df.iloc[len(train):,:].copy()
test.drop(['casual','registered'], axis=1, inplace=True)
display(train.head())
display(train.shape)
display(test.head())
display(test.shape)
# test.shape

,year,month,weekday,hour,day_type,weather,temp,humidity,windspeed,casual,registered,casual_mov_ave,registered_mov_ave
0,2011,1,2,0,1,1,9.84,81,0.0,3.0,13.0,NaN,NaN
1,2011,1,2,1,1,1,9.02,80,0.0,8.0,32.0,NaN,NaN
2,2011,1,2,2,1,1,9.02,80,0.0,5.0,27.0,NaN,NaN
3,2011,1,2,3,1,1,9.84,75,0.0,3.0,10.0,NaN,NaN
4,2011,1,2,4,1,1,9.84,75,0.0,0.0,1.0,NaN,NaN


(10886, 13)

,year,month,weekday,hour,day_type,weather,temp,humidity,windspeed,casual_mov_ave,registered_mov_ave
10886,2011,1,4,0,2,1,10.66,56,26.0027,NaN,NaN
10887,2011,1,4,1,2,1,10.66,56,0.0000,NaN,NaN
10888,2011,1,4,2,2,1,10.66,56,0.0000,NaN,NaN
10889,2011,1,4,3,2,1,10.66,56,11.0014,NaN,NaN
10890,2011,1,4,4,2,1,10.66,56,11.0014,NaN,NaN


(6493, 11)

In [622]:
df.to_csv('output/df_prepro.csv',index=False)

In [623]:
# df = pd.read_csv('output/df_prepro.csv')